## Instruction

All the code is depending on python 3.5, pandas, scikit-learn, json

## Fire up the Library 

In [1]:
import pandas as pd
import numpy as np


## Remove punctuation and set sentiment 

In [2]:
products = pd.read_csv('amazon_baby.csv')

products = products.fillna({'review':''})

def remove_punctuation(text):
    #print(text)
    import string
    instr = string.punctuation
    outstr = " "*len(string.punctuation)
    table = str.maketrans(instr, outstr )
    return text.translate(table) 


#products['review_clean'] = products['review'].apply(str)
products['review_clean'] = products['review'].apply(remove_punctuation)


products = products[products['rating'] != 3]

products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)

In [3]:
#products['review_clean'] = products['review'].apply(remove_punctuation)
len(train)

NameError: name 'train' is not defined

## Get train_matrix 

In [6]:
import json

with open('module-2-assignment-test-idx.json') as json_data:
    d = json.load(json_data)
test = products.iloc[d]
with open('module-2-assignment-train-idx.json') as json_data:
    d1 = json.load(json_data)
train = products.iloc[d1]





from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b') 
# r'\b\w+\b' is the keep word by space. \b is space, \w+ is word
     # Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test['review_clean'])

In [5]:
train_matrix

<133416x57445 sparse matrix of type '<class 'numpy.int64'>'
	with 7431855 stored elements in Compressed Sparse Row format>

I have no idea about the difference of scikit learn, i use the same code on the website,
but it seems like I only get 57745 params, but it should be 121712

## Train the model 

In [4]:
from sklearn.linear_model import LogisticRegression

traget = train['sentiment']

LR_total = LogisticRegression()
result = LR_total.fit(train_matrix, traget)

## Calculate the positive params 

In [5]:
n = 0
for i in result.coef_[0]:
    if i >= 0:
        n = n+1
        
print(n)
print(len(result.coef_[0]))

39463
57445


In [15]:
train_matrix

<133416x57445 sparse matrix of type '<class 'numpy.int64'>'
	with 7431855 stored elements in Compressed Sparse Row format>

In [226]:
sample_test_data = test[10:13]
sample_test_data['review'].iloc[0]

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

In [182]:
sample_test_matrix = vectorizer.transform(sample_test_data['review'])
scores = LR_total.decision_function(sample_test_matrix)
scores

array([  4.96098458,  -3.01485982, -10.69266387])

In [66]:
LR.predict(sample_test_matrix)

array([ 1, -1, -1], dtype=int64)

In [183]:
Proba = 1/(1+np.exp(scores))
Proba

array([ 0.00695728,  0.95324095,  0.99997729])

In [184]:
Proba_model = LR_total.predict_proba(sample_test_matrix)
Proba_model

array([[  6.95728350e-03,   9.93042716e-01],
       [  9.53240947e-01,   4.67590528e-02],
       [  9.99977290e-01,   2.27104229e-05]])

In [185]:
Proba_model_all = LR_total.predict_proba(test_matrix)
print(Proba_model_all[:,0])

[  2.85926408e-01   7.02158647e-07   7.44626277e-02 ...,   2.10178785e-05
   2.53476120e-06   2.33217997e-02]


## Find the most positive review

In [230]:
test['proba'] = np.array(Proba_model_all[:,0])

C:\Users\kinsly\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [231]:
test_AfterSort = test.sort_values(['sentiment','proba'],ascending = False)

In [227]:
test_AfterSort.name[0:20]

83234         Thirsties Hemp Inserts 2 Pack, Small 6-18 Lbs
131271    Motorola MBP36 Remote Wireless Video Baby Moni...
168391        Chicco NextFit Convertible Car Seat, Mystique
129657    Thirsties 3 Pack Boys Fab Doublers Soft Cotton...
140418    Jolly Jumper Arctic Sneak A Peek Infant Car Se...
72270     Baby Trend Expedition Double Jogging Stroller,...
130219    Exergen Temporal Artery Thermometer MODEL# TAT...
13151         Roundabout Convertible Car Seat - Grey Wicker
155995                  Similac SimplySmart Bottle, 4 Ounce
3167          Sunshine Kids Mighty Tite Seat Belt Tightener
171898       Medela 67050 Swing Breast Pump (RECONDITIONED)
154361    AppleCheeks Washable Swim Diaper (Size 2, Mojito)
176830    Spectra Baby USA S1 Hospital Grade Double/sing...
178767    Fisher-Price Ipad Apptivity Seat, Newborn-to-T...
117053               Harmony Kids Standard Rocker, Hot Pink
11352            Fisher-Price Booster Seat, Blue/Green/Gray
30596     Munchkin 2 Pack Fresh Food Fee

In [232]:
test_AfterSort_ascend = test.sort_values(['sentiment','proba'],ascending = True)
test_AfterSort_ascend.name[:20]

11496            Fisher-Price Booster Seat, Blue/Green/Gray
41741     South Shore Furniture, Peak-a-Boo Collection, ...
131329    Motorola MBP36 Remote Wireless Video Baby Moni...
86793     Baby Jogger 2010 City Mini Double Stroller, Bl...
57104                    BabyPlus Prenatal Education System
105860         Joovy Caboose Ultralight Stroller, Blueberry
53935     Baby Einstein Musical Motion Activity Jumper, ...
56909                          Medela Freestyle Breast Pump
110871                     Boppy Prenatal Total Body Pillow
161262    Graco SnugRider Elite Stroller &amp; Car Seat ...
3063                           Graco Pack 'n Play Sheet Tea
110561                 Joovy Nook Highchair, Greenie Fabric
123403       Graco Highback TurboBooster Car Seat, Spitfire
9008                       DEX Products Sound Sleeper SS-01
115375    Summer Infant Carter's Classic Comfort Reclini...
119536    Trend Lab Adjustable Cloth Diaper with Liner, ...
48046     The First Years Learning Curve

In [209]:
test_prediction = LR_total.predict(test_matrix)
test['sentiment_predict'] = test_prediction

C:\Users\kinsly\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [210]:
correct = (len(test[test['sentiment']==test['sentiment_predict']]))
total = (len(test))
accuracy = correct/total
accuracy

0.9321454283657308

## Learn another classifier with fewer words

In [107]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [109]:
vectorizer_word_subset = CountVectorizer(vocabulary=significant_words) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform(train['review_clean'])
test_matrix_word_subset = vectorizer_word_subset.transform(test['review_clean'])

In [187]:
traget = train['sentiment']

LR_subset = LogisticRegression()
result_subset = LR_subset.fit(train_matrix_word_subset, traget)

In [129]:
coef_table = pd.DataFrame(data = result_subset.coef_.T, columns = ['coef'])
coef_table['word'] = significant_words

In [233]:
print(result_subset.intercept_)
coef_table

[ 1.29311199]


,coef,word
0,1.356421,love
1,0.943508,great
2,1.183238,easy
3,0.082776,old
4,0.514632,little
5,1.506878,perfect
6,1.677630,loves
7,0.530188,well
8,0.193079,able
9,0.056467,car


In [171]:
vocab = list(vectorizer.vocabulary_)
sentiment_model_coef_table = pd.DataFrame(data = result.coef_[0], columns = ['coef'])
sentiment_model_coef_table['word'] = vocab
temp = sentiment_model_coef_table.copy()
#temp.loc[significant_words[0]]

In [175]:
temp.index = vocab
temp.loc[significant_words]

,coef,word
love,-2.405787e-06,love
great,6.881617e-03,great
easy,3.098818e-06,easy
old,9.432181e-02,old
little,4.075364e-01,little
perfect,6.325836e-01,perfect
loves,7.040350e-03,loves
well,1.069722e-03,well
able,1.652672e-02,able
car,2.580584e-07,car


In [188]:
simple_prediction = LR_subset.predict(test_matrix_word_subset)
test['simple_predict'] = simple_prediction

C:\Users\kinsly\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [178]:
correct = (len(test[test['sentiment']==test['simple_predict']]))
total = (len(test))
accuracy = correct/total
accuracy

0.8702003839692825

In [190]:
simple_train_prediction = LR_subset.predict(train_matrix_word_subset)
train['simple_predict'] = simple_train_prediction

C:\Users\kinsly\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [195]:
correct_train_subset = (len(train[train['sentiment']==train['simple_predict']]))
train_subset = (len(train))
accuracy_train_subset = correct_train_subset/train_subset
print('accuracy_train_subset',accuracy_train_subset)

accuracy_train_subset 0.8672198237093003


In [198]:
total_train_prediction = LR_total.predict(train_matrix)
train['total_predict'] = total_train_prediction
correct_train_total = (len(train[train['sentiment']==train['total_predict']]))
total_train = (len(train))
accuracy_train_total = correct_train_total/total_train
print('accuracy_train_total',accuracy_train_total)

accuracy_train_total 0.9631378545301913


C:\Users\kinsly\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


## Get the accuracy 

In [222]:
def get_classification_accuracy(model, data, true_labels,dataframe):
    # First get the predictions
    ## YOUR CODE HERE
    prediction = model.predict(data)
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    temp = dataframe.copy()
    temp['predict'] = prediction
    correct = (len(temp[temp[true_labels]==temp['predict']]))
    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    total = (len(dataframe))
    print(correct)
    accuracy = correct/total
    return accuracy
   
    
    
    

In [223]:
train_accuracy = get_classification_accuracy(LR_total, train_matrix, 'sentiment',train)
train_subset_accuracy = get_classification_accuracy(LR_subset, train_matrix_word_subset, 
                                                    'sentiment', train)

In [224]:
test_accuracy = get_classification_accuracy(LR_total, test_matrix, 'sentiment', test)
test_subset_accuracy = get_classification_accuracy(LR_subset, test_matrix_word_subset,
                                                  'sentiment', test)

In [225]:
print('accuracy of total train: ',train_accuracy)
print('accuracy of subset train: ', train_subset_accuracy)
print('accuracy of total test: ', test_accuracy)
print('accuracy of subset test: ', test_subset_accuracy)

accuracy of total train:  0.9631378545301913
accuracy of subset train:  0.8672198237093003
accuracy of total test:  0.9321454283657308
accuracy of subset test:  0.8702003839692825


In [221]:
def get_classification_accuracy(model, data, true_labels,dataframe):
    # First get the predictions
    ## YOUR CODE HERE
    prediction = model.predict(data)
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    temp = dataframe.copy()
    temp['predict'] = prediction
    correct = (len(temp[temp[true_labels]==temp['predict']]))
    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    total = (len(dataframe))
    print(correct)
    accuracy = correct/total
    return accuracy

prediction = LR_total.predict(test_matrix)
temp = test.copy()
temp['predict'] = prediction
correct = (len(temp[temp['sentiment']==temp['predict']]))
print(correct)
get_classification_accuracy(LR_total, test_matrix, 'sentiment',test)

31074
31074


0.9321454283657308

In [236]:
num_positive  = (train['sentiment'] == +1).sum()
num_negative = (train['sentiment'] == -1).sum()
print (num_positive)
print (num_negative)

num_positive/len(train)

112164
21252


0.84070876056844757

In [238]:
test_positive = (test['sentiment'] == +1).sum()
test_negative = (test['sentiment'] == -1).sum()

test_positive / len(test)

0.84278257739380846